In [10]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from keras.models import Sequential
from keras.layers import Dense,Dropout

# --- Configuration ---
# File path for the uploaded dataset
FILE_PATH = 'SCOA_A5.csv'
# The CSV contains multiple stocks. We filter for one to predict future movement.
STOCK_TICKER = 'AAL'

# Step 1: Load stock data from CSV
try:
    data = pd.read_csv(FILE_PATH)
except FileNotFoundError:
    print(f"Error: File not found at {FILE_PATH}")
    exit()

# Filter data for the selected stock ticker
df = data[data['Name'] == STOCK_TICKER].copy()

# Ensure the data is sorted by date for correct shifting
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').reset_index(drop=True)

# Create the prediction target: 1 if the next day's close is higher, 0 otherwise.
# Note: The column names are in lowercase in the CSV (e.g., 'close').
df['Target'] = np.where(df['close'].shift(-1) > df['close'], 1, 0)

# Drop the last row, as its target cannot be calculated (Target will be NaN)
df = df.dropna()

print(f"Data loaded for {STOCK_TICKER}. Total records: {len(df)}")


# Step 2: Feature selection and scaling
# Use the same features as the original script, adjusted for lowercase column names.
features = df[['open', 'high', 'low', 'close', 'volume']]
scaler = MinMaxScaler()
X = scaler.fit_transform(features)
y = df['Target'].values.astype(np.int32) # Ensure target is integer type

# Step 3: Train-test split
# Using shuffle=False is critical for time-series data to maintain chronological order.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Step 4: Build and train ANN model
# Input dimension is 5 (open, high, low, close, volume)
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print("Training model...")
# Training with reduced verbosity for cleaner output
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)
print("Training complete.")

# Step 5: Evaluate model
y_pred_proba = model.predict(X_test, verbose=0)
y_pred = (y_pred_proba > 0.5).astype("int32")

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"\n--- Model Evaluation for {STOCK_TICKER} ---")
print(f"Test Accuracy: {accuracy:.4f}")
print("Confusion Matrix (True Negatives, False Positives, False Negatives, True Positives):")
print(conf_matrix)

# Interpretation of the confusion matrix (optional but helpful)
tn, fp, fn, tp = conf_matrix.ravel()
print(f"\n- True Positives (Correctly predicted UP): {tp}")
print(f"- True Negatives (Correctly predicted DOWN): {tn}")
print(f"- False Positives (Predicted UP, but went DOWN): {fp}")
print(f"- False Negatives (Predicted DOWN, but went UP): {fn}")

Data loaded for AAL. Total records: 1259
Training model...
Epoch 1/100


C:\Users\Laptop\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5244 - loss: 0.6982
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5221 - loss: 0.6920
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5142 - loss: 0.6930
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5079 - loss: 0.6915
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5225 - loss: 0.6907
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5304 - loss: 0.6885
Epoch 7/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5636 - loss: 0.6869
Epoch 8/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5118 - loss: 0.6931
Epoch 9/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5305 - loss: 0.6878
Epoch 10/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5161 - loss: 0.6896
Epoch 11/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5261 - loss: 0.6905
Epoch 12/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5243 - lo

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5171 - loss: 0.6850
Epoch 70/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5323 - loss: 0.6838
Epoch 71/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5280 - loss: 0.6848
Epoch 72/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5439 - loss: 0.6831
Epoch 73/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5294 - loss: 0.6838
Epoch 74/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5223 - loss: 0.6859
Epoch 75/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5430 - loss: 0.6827
Epoch 76/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5286 - loss: 0.6828
Epoch 77/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5386 - loss: 0.6816
Epoch 78/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5327 - loss: 0.6853
Epoch 79/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5728 - loss: 0.6795
Epoch 80/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5

In [ ]:
# -------------------------------------------------------------
# 📘 PROJECT: Stock Movement Prediction using Artificial Neural Network (ANN)
# -------------------------------------------------------------
# This program predicts whether a stock’s next day closing price will
# go UP (1) or DOWN (0) using a feedforward Artificial Neural Network.
# -------------------------------------------------------------
# Libraries used:
#  - numpy, pandas → data handling
#  - sklearn → scaling, splitting, evaluation metrics
#  - keras (Sequential, Dense, Dropout) → building ANN model
# -------------------------------------------------------------

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout

# --- Configuration ---
# File path for dataset and ticker selection
FILE_PATH = 'SCOA_A5.csv'      # dataset file
STOCK_TICKER = 'AAL'           # stock symbol to filter

# -------------------------------------------------------------
# STEP 1: LOAD AND PREPARE THE DATA
# -------------------------------------------------------------
try:
    data = pd.read_csv(FILE_PATH)
except FileNotFoundError:
    print(f"Error: File not found at {FILE_PATH}")
    exit()

# Filter data for selected stock ticker
df = data[data['Name'] == STOCK_TICKER].copy()

# Convert 'date' column to datetime type and sort by date
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').reset_index(drop=True)

# Create the target variable:
# If next day's close price > today's close → 1 (UP), else 0 (DOWN)
df['Target'] = np.where(df['close'].shift(-1) > df['close'], 1, 0)

# Drop the last row because its "next day" value does not exist (NaN)
df = df.dropna()

print(f"Data loaded for {STOCK_TICKER}. Total records: {len(df)}")

# -------------------------------------------------------------
# STEP 2: FEATURE SELECTION AND SCALING
# -------------------------------------------------------------
# Select features: open, high, low, close, and volume
# These are numeric features that influence price movement.
features = df[['open', 'high', 'low', 'close', 'volume']]

# MinMaxScaler scales all feature values between 0 and 1.
# This helps the neural network train faster and avoids dominance
# of larger numeric features (e.g., volume).
scaler = MinMaxScaler()
X = scaler.fit_transform(features)

# Target (output labels)
y = df['Target'].values.astype(np.int32)  # Convert to integer 0/1

# -------------------------------------------------------------
# STEP 3: TRAIN-TEST SPLIT
# -------------------------------------------------------------
# We split 80% data for training and 20% for testing.
# Note: shuffle=False is important for time-series data
# so that we do not mix past and future data.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

# -------------------------------------------------------------
# STEP 4: BUILD ANN MODEL
# -------------------------------------------------------------
# The ANN (Artificial Neural Network) is a collection of layers:
#  - Input layer (5 neurons = 5 features)
#  - Hidden layers (ReLU activations for non-linearity)
#  - Dropout layers to prevent overfitting
#  - Output layer (Sigmoid for binary classification)
#
# Theory:
#  Each neuron computes: z = w*x + b
#  Activation functions:
#     ReLU: f(x) = max(0, x)
#     Sigmoid: σ(x) = 1 / (1 + e^-x)
#
#  Loss function: Binary Cross-Entropy
#     L = - [y*log(p) + (1-y)*log(1-p)]
#  Optimizer: Adam (adaptive gradient method)
#  Backpropagation adjusts weights to minimize loss.

model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))  # Hidden layer 1
model.add(Dropout(0.2))                                               # Drop 20% neurons randomly
model.add(Dense(64, activation='relu'))                               # Hidden layer 2
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))                               # Hidden layer 3
model.add(Dense(1, activation='sigmoid'))                             # Output layer (0-1 probability)

# Compile model:
# - binary_crossentropy: suitable for binary classification
# - adam: adaptive optimizer (fast convergence)
# - accuracy: metric to evaluate performance
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print("Training model...")

# -------------------------------------------------------------
# STEP 5: TRAIN THE MODEL
# -------------------------------------------------------------
# Model learns weights by minimizing loss using backpropagation.
# epochs → how many passes over the entire dataset
# batch_size → how many samples processed before updating weights
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)
print("Training complete.")

# -------------------------------------------------------------
# STEP 6: MODEL EVALUATION
# -------------------------------------------------------------
# Predict probabilities for the test set
y_pred_proba = model.predict(X_test, verbose=0)

# Convert probabilities into binary class (0 or 1)
y_pred = (y_pred_proba > 0.5).astype("int32")

# Accuracy = (TP + TN) / Total
accuracy = accuracy_score(y_test, y_pred)

# Confusion matrix gives TP, TN, FP, FN counts
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"\n--- Model Evaluation for {STOCK_TICKER} ---")
print(f"Test Accuracy: {accuracy:.4f}")
print("Confusion Matrix:")
print(conf_matrix)

# -------------------------------------------------------------
# INTERPRET CONFUSION MATRIX
# -------------------------------------------------------------
# Confusion matrix layout: [[TN, FP], [FN, TP]]
#  - True Positive (TP): predicted UP, actually UP
#  - True Negative (TN): predicted DOWN, actually DOWN
#  - False Positive (FP): predicted UP, actually DOWN
#  - False Negative (FN): predicted DOWN, actually UP
tn, fp, fn, tp = conf_matrix.ravel()

print(f"\n- True Positives (Correctly predicted UP): {tp}")
print(f"- True Negatives (Correctly predicted DOWN): {tn}")
print(f"- False Positives (Predicted UP, but went DOWN): {fp}")
print(f"- False Negatives (Predicted DOWN, but went UP): {fn}")

# -------------------------------------------------------------
# ADDITIONAL THEORY (for viva preparation)
# -------------------------------------------------------------
# ➤ Artificial Neural Network (ANN)
#    - Mimics the human brain’s neuron connections.
#    - Composed of layers of interconnected nodes (neurons).
#    - Each neuron computes a weighted sum of inputs plus bias,
#      applies an activation function, and passes output forward.
#
# ➤ Learning Process
#    - Forward Pass: compute predicted outputs.
#    - Loss Calculation: measure error between predicted and actual.
#    - Backpropagation: propagate error backward, update weights.
#    - Optimizer (Adam): updates weights using gradient descent.
#
# ➤ Why Scaling?
#    - Ensures all features have equal influence.
#    - Improves gradient descent convergence speed.
#
# ➤ Why Dropout?
#    - Prevents overfitting by randomly deactivating neurons.
#
# ➤ Why Sigmoid Output?
#    - Converts output into probability between 0 and 1.
#
# ➤ Why Binary Cross-Entropy Loss?
#    - Measures how close predicted probabilities are to true labels.
#
# ➤ Common Improvements:
#    - Fit scaler only on training data (avoid data leakage).
#    - Use TimeSeriesSplit for better validation.
#    - Add more features like returns, moving averages, RSI, etc.
#    - Try LSTM or GRU for sequential (time-dependent) data.
#    - Use EarlyStopping to avoid overtraining.
#
# -------------------------------------------------------------
# END OF SCRIPT
# -------------------------------------------------------------


In [ ]:
# -------------------------------------------------------------
# INTERPRETATION OF THE TRAINING LOG & EVALUATION (paste as comments)
# -------------------------------------------------------------
# Training log summary:
#  - Training accuracy hovers around ~0.50–0.57 and the training loss stays ~0.68–0.69.
#  - Note: binary cross-entropy for random guessing (p=0.5) ≈ 0.6931 — so loss ≈ 0.69 means
#    the model is often predicting probabilities near 0.5 (i.e. not confident).
#  - The accuracy does not show steady improvement across epochs — it fluctuates.
#
# Test set results (confusion matrix given):
#  [[TN, FP],
#   [FN, TP]] = [[91, 32],
#                [76, 53]]
#  - Total samples = 91 + 32 + 76 + 53 = 252
#  - Test Accuracy = (TP + TN) / Total = (53 + 91) / 252 = 0.5714 (57.14%)
#
# Other useful metrics (from confusion matrix):
#  - Precision (for predicted UP) = TP / (TP + FP) = 53 / 85  ≈ 0.6235
#    -> When model predicts UP, it's correct ~62.4% of the time.
#  - Recall / Sensitivity (for actual UP) = TP / (TP + FN) = 53 / 129 ≈ 0.4109
#    -> The model finds only ~41.1% of actual UP days (many false negatives).
#  - F1-score ≈ 0.4953 (harmonic mean of precision and recall).
#
# Quick verbal interpretation you can say in viva:
#  - "Training loss near 0.69 means the network's outputs are close to 0.5 (uninformative).
#    Accuracy around 50–57% is only slightly better than random guessing for a balanced set.
#    Precision is okay (~62%) but recall is low (~41%), meaning the model is conservative
#    about predicting 'UP' and misses many real UP days (high false negatives)."
#
# -------------------------------------------------------------
# WHY ACCURACY / PERFORMANCE IS LOW — IMPLEMENTATION & MODEL ISSUES
# (short, viva-friendly comment bullets)
# -------------------------------------------------------------
# 1) Target problem is very noisy:
#    - Next-day up/down is inherently noisy and often close to random because markets
#      react to new information. Simple MLP on raw price/volume may not capture the signal.
#
# 2) Model outputs near 0.5 (loss ≈ 0.69) => little learning
#    - Either the model cannot find a useful pattern in the given features, or label noise
#      / bad features make the mapping unlearnable.
#
# 3) Weak / inappropriate features:
#    - Using only [open, high, low, close, volume] for a single-day sample ignores temporal
#      context (momentum, trends, lags). The model has no memory of past days.
#
# 4) Architecture mismatch for time-series:
#    - Plain feedforward MLP treats each day independently and cannot model sequences.
#      Time-dependencies are vital in stock data.
#
# 5) Possible data issues:
#    - Label noise, missing values, incorrect preprocessing or scaling mistakes (e.g., fitting
#      scaler on whole dataset leaks info or produces unstable transforms).
#
# 6) Insufficient feature engineering:
#    - No returns, no lagged features, no technical indicators (moving averages, RSI, etc.).
#
# 7) Poor training regime:
#    - No validation set / early stopping -> may be training longer than necessary or not
#      tuning hyperparameters. Also no reproducibility (random seeds).
#
# 8) Evaluation metric choice:
#    - Accuracy alone can be misleading. The model shows asymmetric errors (low recall).
#
# 9) Overfitting vs underfitting:
#    - The near-constant high loss suggests underfitting or that the model can't extract signal.
#      Dropout may be excessive given the small feature set, or the model is not the right type.
#
# 10) Class threshold and calibration:
#    - Using 0.5 threshold may not be optimal — probabilities might be poorly calibrated.
#
# -------------------------------------------------------------
# SUGGESTED FIXES / IMPROVEMENTS (prioritised & practical — paste into code as comments)
# -------------------------------------------------------------
# A. Quick fixes to check first
#    1. Check scaler leakage:
#       - Fit scaler on X_train only, then transform X_test. (Avoid fitting on whole X before split.)
#         Example:
#           X_train_raw, X_test_raw, y_train, y_test = train_test_split(features, y, test_size=0.2, shuffle=False)
#           scaler = MinMaxScaler()
#           X_train = scaler.fit_transform(X_train_raw)
#           X_test  = scaler.transform(X_test_raw)
#
#    2. Add a proper validation set / use TimeSeriesSplit for hyperparameter tuning.
#    3. Add EarlyStopping callback (monitor val_loss) to stop training when model stops improving.
#
# B. Feature engineering (high impact)
#    1. Use returns instead of raw prices:
#         return_t = (close_t - close_{t-1}) / close_{t-1}
#    2. Add lag features (window of past N days): e.g., last 5–20 days returns or closes.
#    3. Add technical indicators: moving averages, momentum, RSI, MACD, Bollinger Bands, volatility.
#    4. Use volume-normalized features or log(volume) if skewed.
#
# C. Model / architecture changes
#    1. Use sequence models when you include temporal windows:
#       - LSTM / GRU / 1D-CNN or Transformer for time-series data. These capture temporal patterns.
#       - Example approach: for each sample, feed a matrix shape (window_size, features).
#    2. If staying with MLP, provide windowed features (flatten last N days) rather than single day.
#
# D. Training & evaluation improvements
#    1. Use class weights or resampling if classes are imbalanced.
#    2. Try different thresholds and evaluate ROC-AUC, precision/recall curves, F1 — not only accuracy.
#    3. Perform hyperparameter search (layers, neurons, learning rate, batch_size).
#    4. Use model checkpointing and save best model based on validation metric.
#    5. Set random seeds (numpy, tensorflow) for reproducibility.
#
# E. Robustness & real-world considerations
#    1. Use walk-forward validation (rolling window) to simulate real trading conditions.
#    2. Test across market regimes (bull, bear) and ensure model generalizes.
#    3. Work with risk-adjusted metrics or strategy backtest (include transaction costs).
#
# F. Baselines & diagnostics
#    1. Always compare to simple baselines:
#       - Always predict majority class
#       - Logistic regression on engineered features
#       - Decision tree / random forest
#    2. If a simple baseline matches your ANN, the ANN is not adding value — go back to feature engineering.
#
# G. Calibration & threshold tuning
#    - Compute predicted probabilities on validation set, plot ROC, choose threshold that balances precision/recall for your objective.
#
# -------------------------------------------------------------
# CONCISE VIVA LINES (say these if asked "Why is accuracy low and what will you do?")
# -------------------------------------------------------------
# - "Accuracy is low because next-day price direction is noisy and the model currently has weak features 
#    (single-day raw prices) and no temporal context. Training loss ≈0.69 indicates the network often outputs 
#    probabilities near 0.5 (uninformative)."
# - "I will fix preprocessing (fit scaler on train), add lagged features and indicators, use a time-series
#    model (LSTM or windowed MLP), introduce proper validation (TimeSeriesSplit), and tune hyperparameters."
#
# -------------------------------------------------------------
# END OF COMMENT BLOCK
# -------------------------------------------------------------
